In [41]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Imputer

from sklearn.metrics import f1_score

ModuleNotFoundError: No module named 'graphviz'

In [2]:
train = pd.read_csv('train.csv')

In [3]:
train['Sex'] = train['Sex'].replace({'male' : 0, 'female' : 1})

In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S


In [5]:
train.describe()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,0.352413,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,0.477990,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,0.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,1.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200


In [6]:
train = train[['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Age', 'Survived']]

In [7]:
imputer = Imputer()

In [8]:
vaiq = train.drop('Survived', axis=1)

# eita
Vamos escrever algo aq

In [9]:
vaiq = pd.DataFrame(imputer.fit_transform(vaiq), index=vaiq.index, columns=vaiq.columns)

In [10]:
vaiq.describe()

,Pclass,Sex,SibSp,Parch,Fare,Age
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,2.308642,0.352413,0.523008,0.381594,32.204208,29.699118
std,0.836071,0.477990,1.102743,0.806057,49.693429,13.002015
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.420000
25%,2.000000,0.000000,0.000000,0.000000,7.910400,22.000000
50%,3.000000,0.000000,0.000000,0.000000,14.454200,29.699118
75%,3.000000,1.000000,1.000000,0.000000,31.000000,35.000000
max,3.000000,1.000000,8.000000,6.000000,512.329200,80.000000


# Que diabos é uma Arvore de Decisão (*Decision Tree*)?

Fontes:

- Python Machine Learning,Raschka, S., isbn 9781783555147, https://books.google.com.br/books?id=GOVOCwAAQBAJ
 - [Primeira edição](https://www.packtpub.com/big-data-and-business-intelligence/python-machine-learning)
 - [Segunda edição](https://www.packtpub.com/big-data-and-business-intelligence/python-machine-learning-second-edition)


Uma árvore de decisão é um algoritmo que divide a base de dados na variável (sklearn divide sempre em duas partes, pois é mais fácil) que resulta em maior ganho de informação. O Ganho de Informação $GI$ é definido a seguir:
$$ GI(D_p, f) = I(D_p) - \sum^m_{j = 1}{\frac{N_j}{N_p}I(D_j)}$$
Onde $D_p$ e $D_j$ são os dados do nó pai e júnior (filho), $I$ é a medida de impuridade, $N_p$ é o número total de amostras do pai e $N_j$ o número de amostras do $j$-ésimo júnior (filho).

Normalmente se usa usa uma de três medidas de impureza: **Impureza de Gini** ($I_G$), **Entropia** ($I_H$) e **erro de classficação** ($I_E$).

In [11]:
vaiq.head(3)

,Pclass,Sex,SibSp,Parch,Fare,Age
0,3.0,0.0,1.0,0.0,7.2500,22.0
1,1.0,1.0,1.0,0.0,71.2833,38.0
2,3.0,1.0,0.0,0.0,7.9250,26.0


In [45]:
?train_test_split

In [13]:
x_train, x_test, y_train, y_test= train_test_split(vaiq, train['Survived'], test_size=0.3, random_state=42)

In [14]:
?DecisionTreeClassifier

In [15]:
arvore = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=42)

In [16]:
arvore.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [42]:
?export_graphviz

In [43]:
export_graphviz(arvore, out_file='arvore_titanic.dot', label=['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Age'])

In [44]:
!dot -Tpng arvore_titanic.dot -o arvore_titanic.png

'dot' is not recognized as an internal or external command,
operable program or batch file.


In [20]:
previsao = arvore.predict(x_test)

In [21]:
(previsao == y_test).sum()/(len(y_test))

0.80970149253731338

In [22]:
len(y_test)

268

In [23]:
f1_score(y_test, previsao)

0.75362318840579712

# Random Forest

In [24]:
forest = RandomForestClassifier(n_estimators=30, max_depth=3, random_state=42, max_features=2)

In [25]:
floresta = forest.fit(x_train, y_train)

In [26]:
floresta

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features=2, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [27]:
prevendo = floresta.predict(x_test)

In [28]:
(prevendo == y_test).sum()/(len(y_test))

0.79104477611940294

In [29]:
f1_score(y_test, prevendo)

0.69565217391304357

In [30]:
floresta_completa = forest.fit(vaiq, train['Survived'])

In [31]:
test_kaggle = pd.read_csv('test.csv')

In [32]:
passenger_id = test_kaggle['PassengerId']

In [33]:
test_kaggle = test_kaggle[['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Age']].replace({'male' : 0, 'female' : 1})

In [34]:
test_kaggle = pd.DataFrame(imputer.transform(test_kaggle), index=test_kaggle.index, columns=test_kaggle.columns)

In [35]:
envio = floresta_completa.predict(test_kaggle)

In [36]:
submission = pd.DataFrame({'PassengerId': passenger_id, 'Survived':envio})

In [37]:
submission.index = submission['PassengerId']

In [38]:
submission = submission.drop('PassengerId', axis=1)

In [39]:
submission.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


In [47]:
submission.to_csv('sub.csv', index=False)